In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('~/Dropbox/Element/Phase2/TradesJan22.csv')

In [3]:
df.shape

(1488, 244)

In [4]:
# Drop zero fill rows
df = df[df['fillQuantity'] > 0]

In [5]:
# Deal with apparent float precision issues to return true penny prices
round_cols = [col for col in df.columns[3:] if df[col].dtype=='float64']
round_cols = [col for col in round_cols if ('Vol' not in col) and ('Prob' not in col) and ('Mark' not in col)]
for col in round_cols:
    df[col] = df[col].apply(lambda x: round(x, 2))

In [6]:
# Convert Date Cols and add in microsecond cols
def col_to_time(col):
    df[col] = df[col].apply(pd.to_datetime)
    df[col] = df[col].dt.tz_localize('America/Chicago').dt.tz_convert('America/New_York')
    s = df[col+'_us'].apply(pd.Timedelta, unit='micros')
    df[col] = df[col] + s
    
for col in ['childDttm', 'fillTransactDttm']:
    col_to_time(col)

df['parentDttm'] = df['parentDttm'].apply(lambda s: pd.to_datetime(s))
df['parentDttm'] = df['parentDttm'].dt.tz_localize('America/Chicago').dt.tz_convert('America/New_York')

In [7]:
parents = df['baseParentNumber'].unique()
parents

array([1055557743596388357, 1136625872087341720,  763530789521261100])

In [8]:
order1 = df[df['baseParentNumber'] == parents[0]] # buying $15m of SPY to hedge overnight delta (40,000 shrs)
order2 = df[df['baseParentNumber'] == parents[1]] # sell SPX Mar 3850 C autohedge AlphaVwap2pct (1000 ctr)
order3 = df[df['baseParentNumber'] == parents[2]] # autohedges (500,782 shrs)

In [9]:
for o in [order1, order2, order3]:
    print(o['parentDttm'].unique())

<DatetimeArray>
['2021-01-22 10:10:33-05:00']
Length: 1, dtype: datetime64[ns, America/New_York]
<DatetimeArray>
['2021-01-22 12:29:06-05:00', '2021-01-22 13:38:05-05:00']
Length: 2, dtype: datetime64[ns, America/New_York]
<DatetimeArray>
['2021-01-22 12:29:22-05:00', '2021-01-22 12:29:36-05:00',
 '2021-01-22 12:35:56-05:00', '2021-01-22 12:44:40-05:00',
 '2021-01-22 12:56:48-05:00', '2021-01-22 13:04:26-05:00',
 '2021-01-22 13:10:52-05:00', '2021-01-22 13:21:48-05:00',
 '2021-01-22 13:42:43-05:00', '2021-01-22 13:42:54-05:00',
 '2021-01-22 13:43:04-05:00', '2021-01-22 13:43:18-05:00',
 '2021-01-22 13:43:24-05:00', '2021-01-22 13:43:52-05:00',
 '2021-01-22 13:43:53-05:00', '2021-01-22 13:44:20-05:00',
 '2021-01-22 13:44:22-05:00', '2021-01-22 13:44:28-05:00',
 '2021-01-22 13:44:41-05:00', '2021-01-22 13:44:43-05:00',
 '2021-01-22 13:45:05-05:00', '2021-01-22 13:45:06-05:00',
 '2021-01-22 13:45:08-05:00', '2021-01-22 13:45:52-05:00']
Length: 24, dtype: datetime64[ns, America/New_York]


In [10]:
def calc_fil_pctSpd(df):
    # Returns average fill as % of spread at arrival time.  0 = Bid, 0.5 = Mid, 1 = Ask
    s = (df['fillPrice'] - df['fillBid']) / (df['fillAsk'] - df['fillBid']) * df['fillQuantity']
    return s.sum() / df['fillQuantity'].sum()

for o in [order1, order2, order3]:
    print(f'{calc_fil_pctSpd(o):.2%}')
    
# Ok.  'Active taker algos' only ever hit the bid or offer - they just think they're smart about when
# The 'patient' algos seem to do very little making

99.67%
0.00%
99.93%


In [11]:
# Compare delta adjusted fill vs arrival mid and mark
arrival_ul = (order3.loc[order3.index[0], 'parentAsk'] + order3.loc[order3.index[0], 'parentBid'])/2
arrival_idx = (order2.loc[order2.index[0], 'parentUAsk'] + order2.loc[order2.index[0], 'parentUBid'])/2
arrival_mid = (order2.loc[order2.index[0], 'parentAsk'] + order2.loc[order2.index[0], 'parentBid'])/2
arrival_mark = order2.loc[order2.index[0], 'parentMark']

print(arrival_ul)
print(arrival_idx)
print(arrival_mid)
print(arrival_mark)

383.695
3848.785
105.8
105.7586594


In [12]:
# Delta adjust option fill by actual SPY execution
avg_ul = (order3['fillPrice'] * order3['fillQuantity']).sum() / order3['fillQuantity'].sum()
avg_opt = (order2['fillPrice'] * order2['fillQuantity']).sum() /  order2['fillQuantity'].sum() 
delta = order2.loc[order2.index[0], 'fillDe']
delta_exec = order3['fillQuantity'].sum() / (order2['fillQuantity'].sum() * 100 * 10)
print(delta, delta_exec)
arrival_opt_adj = avg_opt + delta * (arrival_ul - avg_ul)
print(avg_opt)
print(arrival_opt_adj)
print (arrival_opt_adj - arrival_mid)
print (arrival_opt_adj - arrival_mark)

0.5 0.500782
105.38909999999998
105.4146409439636
-0.38535905603639264
-0.3440184560363946


In [13]:
# Compare vs contract vega
vega = order2.loc[order2.index[0], 'fillVe']
print(vega)
print((arrival_opt_adj - arrival_mid) / vega)
print((arrival_opt_adj - arrival_mark) / vega)
# so less than .1 vol discount vs. mark

6.04
-0.0638011682179458
-0.05695669801927063


In [14]:
# Compare vs (*not* fill-weighted) average spread
spd_sum = (order2['fillAsk'] - order2['fillBid']).sum()
half_spd = 0.5 * spd_sum / order2.shape[0]
print(half_spd)
print((arrival_opt_adj - arrival_mid) / half_spd)
print((arrival_opt_adj - arrival_mark) / half_spd)

# This doesn't look good.  It means we crossed spread on both options and delta
# so ended up selling below the bid

# Although - we only capture the bid-offer when it trades.  That might be when it's unusually tight

# Also, this analysis doesn't take into account whether vol went down over the period.
# We could use the ATM vols provided to investigate that

0.15489130434782516
-2.4879321512525148
-2.221031435463052


In [15]:
# Calculate theoretical delta adj as if exactly hedging each fill
# pctCross is how much of spread to assume is crossed. 0 = bid, 1 = ask

pctCross = 0.5
ul_prices = (1 - pctCross) * order2['fillUBid'] + pctCross * order2['fillUAsk']
avg_theo_ul = (ul_prices * order2['fillQuantity']).sum() / order2['fillQuantity'].sum()
arrival_opt_theo_adj = avg_opt + delta * (arrival_idx - avg_theo_ul)
arrival_opt_theo_adj

# Interesting.  It seems that SPY is so tight and hedging so fast that it makes no difference
# The execution price is dominated by the option fills

105.6422275000001

In [16]:
# For the stock trades, how many were making vs taking?

def calc_post_pct(df):
    post_shrs = df.loc[df['childOrderHandling']=='PostLimit', 'fillQuantity'].sum()
    return post_shrs / df['fillQuantity'].sum()

print(f'{calc_post_pct(order1):.1%}') # TwapAlpha
print(f'{calc_post_pct(order3):.1%}') # Mix of AlphaVwap2pct and SpdrAuto

4.4%
0.7%


In [17]:
# An observation:
half_spd_SPY = 0.5 * (order1['fillAsk'] - order1['fillBid']).sum() / order1.shape[0]
opt_cost = half_spd * order2['fillQuantity'].sum() * 100
spy_cost = half_spd_SPY * order3['fillQuantity'].sum()
print(f'SPY hedge cost is {spy_cost / (spy_cost + opt_cost):.0%} of total cost.')

SPY hedge cost is 17% of total cost.
